In [2]:
import pandas as pd

# Load the dataset from Hugging Face's raw CSV URL
csv_url = "https://huggingface.co/datasets/IDEALLab/power_electronics_v0/resolve/main/dataset_v0_1.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_url)

# Display the first few rows
print(df.head())

# Save it locally if needed
df.to_csv("./data/power_electronics_v0_1.csv", index=False)


HTTPError: HTTP Error 404: Not Found

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Compute log(r + 1e-8)
log_r = np.log(df[' r'])

# Histogram
plt.figure(figsize=(7, 5))
plt.hist(log_r, bins=200, alpha=0.7)
plt.xlabel("log(r)")
plt.ylabel("Count")
plt.title("Histogram of log(r)")
plt.show()


In [1]:
# better cause circuit as categorical values so dimless i think not needed
!python ./engiopt/shape2shape_leastV_vae_hybsm_nodatap.py \
    --data_dir "../EngiOpt/data" \
    --data_input "power_electronics_v0_1.csv" \
    --init_col "" \
    --opt_col "" \
    --target_col "r" \
    --log_target \
    --params_cols '["C1","C2","C3","C4","C5","C6","L1","L2","L3","T1", "g"]' \
    --l2_lambda 1e-6\
    --strip_column_spaces \
    --flatten_columns "[]" \
    --lambda_lv 1e-2 \
    --learning_rate 1e-3 \
    --lr_decay 0.99 \
    --activation "tanh" \
    --no-structured \
    --hidden_layers 5 \
    --hidden_size 256 \
    --latent_dim 8 \
    --n_epochs 10 \
    --batch_size 16 \
    --patience 5 \
    --scale_target \
    --track \
    --wandb_project "shape2shape_leastV_vae_hybsm_pe_test2"\
    --seed 17 \
    --save_model \
    --model_output_dir "my_models" \
    --test_model

wandb: Currently logged in as: smassoudi (smassoudi-eth-z-rich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.5
wandb: Run data is saved locally in /Users/smassoudi/Library/CloudStorage/OneDrive-ETHZurich/Code_Collaborative/EngiOpt/wandb/run-20250325_141440-13iej2s8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run power_electronics_v0_1.csv__2025-03-25-14-14-40
wandb: ⭐️ View project at https://wandb.ai/smassoudi-eth-z-rich/shape2shape_leastV_vae_hybsm_pe_test2
wandb: 🚀 View run at https://wandb.ai/smassoudi-eth-z-rich/shape2shape_leastV_vae_hybsm_pe_test2/runs/13iej2s8
Using device: mps
         C1        C2        C3        C4  ...   T1         g         r          e
0  0.000001  0.000001  0.000001  0.000001  ...  0.1  0.440126  0.914354  -3.903844
1  0.000001  0.000001  0.000001  0.000001 

In [11]:
from engiopt.model_pipeline import ModelPipeline
import engiopt.model_pipeline
import sys
sys.modules["model_pipeline"] = engiopt.model_pipeline
import pandas as pd
import numpy as np
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# 1) Load pipeline
pipeline = ModelPipeline.load("my_models/final_pipeline_power_electronics_v0_1.csv__2025-03-25-14-14-40.pkl")
#pipeline.to_device("mps")  # or "cuda" or "cpu"

# 2) Prepare new raw data in a DataFrame
# The DataFrame must include all the raw columns expected based on your training config:
# For example, if you used columns "mach", "reynolds" for parameters and 
# columns starting with "initial_design_" for the initial shape, etc.
data_path = "../EngiOpt/data/power_electronics_v0_1.csv"
raw_data = pd.read_csv(data_path)

# 3) Predict
# Since the pipeline now expects raw input, simply pass the DataFrame.
y_pred = pipeline.predict(raw_data, batch_size=64, device=device)
print("Predictions:", y_pred)

# 4) Evaluate (if you have ground-truth values)
# The evaluate method also expects raw data.
y_true = raw_data[' r']  # For example, substitute with your actual ground-truth array
metrics = pipeline.evaluate(raw_data, y_true, batch_size=64, device=device, 
                            metrics=["mse", "rmse", "rel_err", "mae"])
print("Evaluation metrics:", metrics)


Using device: mps
[DataPreprocessor] Applied log-transform to r
Predictions: [0.8526422  1.0241786  1.3535485  ... 0.25831938 0.2882559  0.33875072]
[DataPreprocessor] Applied log-transform to r
Evaluation metrics: {'mse': 295.5778392223087, 'rmse': 17.192377358070893, 'rel_err': 0.698201411314683, 'mae': 1.9573965817205756}


In [15]:
y_pred

array([0.8526422 , 1.0241786 , 1.3535485 , ..., 0.25831938, 0.2882559 ,
       0.33875072], dtype=float32)

In [16]:
y_true

0       0.914354
1       1.152453
2       1.521469
3       2.190335
4       3.205281
          ...   
4603    0.265396
4604    0.438262
4605    0.419580
4606    0.271920
4607    0.067190
Name:  r, Length: 4608, dtype: float64